# ♟️ Modell elleni sakk (folyamatos játék loop-pal)
Ez a notebook lehetővé teszi, hogy ember játszon az MLP Mixer modell ellen egy ciklusban, grafikus sakktáblán keresztül.

In [ ]:
import chess
import chess.svg
from IPython.display import SVG, display
import torch
from MLP_chess import MixerPolicyValueHead, mixer, fen_to_tensor

model = MixerPolicyValueHead(mixer)
model.load_state_dict(torch.load("mixer_policyvalue_iter5.pt"))
model.eval()

board = chess.Board()

def show():
    display(SVG(chess.svg.board(board)))

def predict_best_move(model, board):
    with torch.no_grad():
        x = torch.tensor(fen_to_tensor(board.fen()), dtype=torch.float32).unsqueeze(0)
        policy_logits, _ = model(x)
        probs = torch.softmax(policy_logits.view(1, 64 * 64), dim=1).view(64, 64)
        legal_moves = list(board.legal_moves)
        scores = {m: probs[m.from_square][m.to_square].item() for m in legal_moves}
        return max(scores, key=scores.get) if scores else None

show()

In [ ]:
# ♻️ Lépésenkénti ciklus
while not board.is_game_over():
    move_input = input("Your move (e.g. e2e4): ")
    try:
        move = chess.Move.from_uci(move_input)
        if move in board.legal_moves:
            board.push(move)
            show()
        else:
            print("Illegal move.")
            continue
    except:
        print("Invalid format.")
        continue

    if board.is_game_over():
        break

    model_move = predict_best_move(model, board)
    print(f"Model plays: {model_move}")
    board.push(model_move)
    show()

print("\nGame Over:", board.result())